Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [187]:
NAME = "Zhiqi Chen"
COLLABORATORS = ""

---

# Homework 5: Data Tables

### © Luca de Alfaro, 2020, [CC-BY_NC License](http://creativecommons.org/licenses/by-nc-nd/4.0/).

For how to work on this homework assignment, please refer to the instructions posted on Canvas. 

There are 5 questions in the homework; each question has one or more test cells. 

## Submission

[Please submit to this Google Form](https://docs.google.com/forms/d/e/1FAIpQLSfi_wJKdT0Z2mJyuzMhk8mfhnNpdZE0Sh2kftQSMVHP0EkIUg/viewform?usp=sf_link).

Deadline: Thursday October 29, 11pm (check on Canvas for updated information).

The importance of data science cannot be underestimated.  The data sets used in data science often come as tables, with rows representing data items, and columns representing the attribute of the data items. 
Packages such as [Pandas](https://pandas.pydata.org/) and [Datatable Frames](https://datatable.readthedocs.io/en/latest/quick-start.html) help process and visualize such tabular data efficiently. 

Perhaps you think that implementing one such tabular computation package is a major effort.  This is very much true, if extreme efficiency, and perhaps the ability to use parallel processing and out-of-memory storage are your goals. 
However, implementing a basic (yet efficient) table processing framework is not very difficult. 
We shall do just that in this notebook, and it will be an instructive journey. 

We will implement Tables, which are similar to [Pandas Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) and to [Datatable Frames](https://datatable.readthedocs.io/en/latest/quick-start.html). 
A Table is ... a table, whose columns have labels that are usually strings (but can be any hashable type), and whose rows have integer indices.  A Table is used to represent data as it may appear in a csv file or a spreadsheet.
Unlike in Datatable, we do not assume that all values in a column be of the same type.  In the package, we will strive for simplicity of use and generality, rather than efficiency. 

To have an idea of where we are aiming, it is strongly recommended that you read the [_10 minutes to Pandas_](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) introduction to Pandas.  If nothing else, Pandas is a very well known and much used tool for data analysis, and knowing roughly what it does will be instructive in its own right. 



## A Quick Tour of NumPy and Pandas

Let's give you a quick tour of NumPy and Pandas.

### Numpy

[NumPy](https://numpy.org/) is a tool for numerical computation that is very commonly used in Python: indeed, NumPy is one of the main reasons why Python is becoming the most used language for data science. 
Numpy also has a "sister package" [SciPy](https://www.scipy.org/), which contains more advanced scientific functions; we will not discuss SciPy here. 

In NumPy, you can create arrays and matrices.  You can then combine these matrices with operators such as $+$, $-$, $/$, and so forth, which operate element-wise on the arrays and matrices. 


In [188]:
import numpy as np

a = np.array([1, 2, 3, 4])
b = np.array([4, 3, 2, 5])
a + b


array([5, 5, 5, 9])

Matrices are equally easy:

In [189]:
m = np.random.random((3, 4)) # Generates a random 3 x 4 matrix
m


array([[0.17464529, 0.46737129, 0.12889631, 0.50007685],
       [0.72295524, 0.33865044, 0.85436468, 0.16865137],
       [0.44312607, 0.35634969, 0.9015801 , 0.4778895 ]])

You can then do matrix multiplication... 

In [190]:
np.matmul(m, a)


array([3.49638421, 4.63795566, 5.77212375])

... and you can easily compute average, max, min, median, and more, of arrays. 

In [191]:
np.average(a)


2.5

You can do much more; so much more, in fact, that nobody starts out by studying all that NumPy can do.  A good rule of thumb is that, if you need something that's even remotely standard, it's likely to be available in NumPy; just search and you will find it. 

### Pandas

Pandas is a tool for processing data in tabular form.  Think at it like a spreadsheet processor, except one that can quickly read spreadsheets with millions of lines, and perform useful data visualizations, aggregate computations, and statistical analysis on them.  Pandas is widely used in data science to preprocess and explore datasets. 

Let's give it a try.  Pandas is imported traditionally with the pd name.  You can easily import data from csv, even if the data comes from a URL.  This dataset is the [US Census Demographic Data dataset by MuonNeutrino on Kaggle](https://www.kaggle.com/muonneutrino/us-census-demographic-data?select=acs2017_county_data.csv). 

In [192]:
import pandas as pd

ACS_URL = "https://storage.googleapis.com/lucadealfaro-share/acs2017_county_data.csv"
df = pd.read_csv(ACS_URL)


We have imported the dataset into `df`, which is a Pandas _dataframe_.  Let's display it. 

In [193]:
df


,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,Puerto Rico,Vega Baja Municipio,54754,26269,28485,96.7,3.1,0.1,0.0,0.0,0.0,42838,18900,1219,10197,576,43.8,49.4,28.6,20.2,25.9,11.1,14.2,92.0,4.2,0.9,1.4,0.6,0.9,31.6,14234,76.2,19.3,4.3,0.2,16.8
3216,72147,Puerto Rico,Vieques Municipio,8931,4351,4580,95.7,4.0,0.0,0.0,0.0,0.0,7045,16261,2414,11136,1459,36.8,68.2,20.9,38.4,16.4,16.9,7.3,76.3,16.9,0.0,5.0,0.0,1.7,14.9,2927,40.7,40.9,18.4,0.0,12.8
3217,72149,Puerto Rico,Villalba Municipio,23659,11510,12149,99.7,0.2,0.1,0.0,0.0,0.0,18053,19893,1935,10449,1619,50.0,67.9,22.5,21.2,22.7,14.1,19.5,83.1,11.8,0.1,2.1,0.0,2.8,28.4,6873,59.2,30.2,10.4,0.2,24.8
3218,72151,Puerto Rico,Yabucoa Municipio,35025,16984,18041,99.9,0.1,0.0,0.0,0.0,0.0,27523,15586,1467,8672,702,52.4,62.1,27.7,26.0,20.7,9.5,16.0,87.6,9.2,0.0,1.4,1.8,0.1,30.5,7878,62.7,30.9,6.3,0.0,25.4


Let's compute the average income. 

In [194]:
df["Income"].mean()


48994.966770186336

However, the above is the average of the income of the _counties_, not the average income of the _people_: the two are not the same, as the population is not the same for all counties. 
How can we compute the average income of _people_?  There may be more elegant ways, but we can do as follows.  First, we compute the total number of people. 

In [195]:
total_people = df["TotalPop"].sum()
total_people


324473370

That's 324 million people, so it's reasonable. 
Now, we add a new column `CountyIncome` to the dataframe, equal to the product of the average income in a county, by the county population.  The column thus corresponds to the total income of people in the county. 

In [196]:
df["CountyIncome"] = df["TotalPop"] * df["Income"]


Let's compute the average income. 

In [197]:
df["CountyIncome"].sum() / df["TotalPop"].sum()


59864.95225865839

The number is in the right ballpark. 

We can also look around.  Which counties are the wealthiest, measured as average income of their residents? 

In [198]:
df.nlargest(10, "Income")


,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,CountyIncome
2872,51107,Virginia,Loudoun County,374558,185575,188983,13.5,57.7,7.2,0.2,17.1,0.1,231130,129588,2111,50456,581,3.7,3.6,57.3,13.2,19.9,4.5,5.1,77.2,9.2,3.7,1.7,1.1,7.0,33.7,201528,78.7,16.8,4.4,0.1,3.8,48538222104
2848,51059,Virginia,Fairfax County,1142004,565593,576411,16.2,51.7,9.3,0.1,18.8,0.0,724787,117515,1314,52976,419,6.1,7.9,56.6,14.7,18.4,5.7,4.7,71.1,9.4,9.6,1.9,1.7,6.3,32.1,609859,73.6,20.8,5.5,0.1,4.5,134202600060
1205,24027,Maryland,Howard County,312495,152843,159652,6.5,54.1,18.1,0.2,17.3,0.0,212310,115576,1799,51045,772,5.2,5.8,60.9,11.8,18.1,4.1,5.1,81.2,7.2,3.8,1.0,1.1,5.7,30.9,167493,73.0,22.7,4.2,0.1,4.2,36116922120
2925,51610,Virginia,Falls Church city,13843,6801,7042,10.6,72.0,4.5,0.0,9.3,0.0,9598,114795,7917,65510,4014,2.9,2.4,67.3,11.7,16.0,2.1,2.9,60.4,7.6,20.1,2.4,2.8,6.7,28.9,7672,64.6,28.5,6.9,0.0,4.2,1589107185
2826,51013,Virginia,Arlington County,229534,114403,115131,15.5,62.3,8.5,0.3,10.2,0.1,162937,112138,2139,67061,879,8.2,10.0,69.0,10.5,13.8,4.1,2.7,53.4,6.2,26.0,5.0,2.8,6.6,28.6,143798,70.8,24.9,4.2,0.2,3.2,25739483692
262,8035,Colorado,Douglas County,320940,159804,161136,8.3,83.3,1.1,0.1,4.4,0.1,222455,111154,1484,48725,653,3.6,3.8,53.7,11.6,25.0,4.6,5.0,78.0,6.7,1.9,1.0,1.4,11.0,27.6,169625,83.3,11.6,5.1,0.1,3.3,35673764760
1783,34019,New Jersey,Hunterdon County,125717,62901,62816,6.2,86.3,2.5,0.1,3.8,0.0,95691,110969,2182,54200,1457,4.5,4.9,51.7,12.9,23.2,6.7,5.5,80.7,5.5,2.6,1.8,0.7,8.7,33.9,66382,79.5,13.7,6.6,0.1,4.4,13950689773
1810,35028,New Mexico,Los Alamos County,18031,9135,8896,17.2,72.9,0.3,1.0,6.0,0.0,13223,110190,6543,52125,2359,5.1,5.4,67.7,12.6,13.0,4.2,2.5,76.5,9.9,3.3,3.9,3.6,2.9,15.7,9122,61.6,34.3,3.8,0.3,3.7,1986835890
1787,34027,New Jersey,Morris County,498847,244480,254367,13.0,72.0,3.0,0.0,10.1,0.0,354414,107034,1215,53491,803,4.6,4.8,51.0,12.9,23.9,5.3,6.9,79.2,6.8,5.4,2.0,0.9,5.6,30.9,262461,82.9,11.6,5.4,0.1,5.2,53393589798
2924,51600,Virginia,Fairfax city,23580,11621,11959,16.9,57.9,4.9,0.1,16.0,0.1,15316,106870,5914,46489,2037,7.8,6.2,53.0,18.1,18.6,6.8,3.6,69.6,7.6,10.1,4.5,1.1,7.2,31.2,12834,74.7,20.8,4.3,0.1,4.1,2519994600


I wonder what's up with Virgina!  Perhaps patricians and plebeians live separately there?  No county in California is in the top 10. 

Let's now compute the top 10 counties according to the total incomes of their residents.  This gives us a measure of the "purchasing power" of each county. 

In [199]:
df.nlargest(10, "CountyIncome")


,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,CountyIncome
204,6037,California,Los Angeles County,10105722,4979641,5126081,48.4,26.5,7.9,0.2,14.3,0.2,6218279,61015,262,30798,129,17.0,24.0,36.4,19.0,24.1,7.6,12.8,73.7,9.6,6.3,2.7,2.3,5.3,30.9,4805817,79.3,11.2,9.3,0.2,7.8,616600627830
610,17031,Illinois,Cook County,5238541,2540704,2697837,25.1,42.7,23.4,0.1,6.9,0.0,3538445,59426,380,33722,200,15.9,23.0,39.3,17.9,23.6,6.0,13.1,61.7,8.1,18.9,4.4,2.4,4.5,32.9,2521437,84.2,11.1,4.6,0.1,8.7,311305537466
2623,48201,Texas,Harris County,4525519,2251060,2274459,42.2,30.6,18.5,0.2,6.8,0.1,2605043,57791,503,30856,222,16.8,25.0,35.4,17.4,22.9,11.5,12.8,79.3,10.9,2.7,1.5,1.9,3.7,28.9,2180392,83.4,9.8,6.6,0.2,6.4,261534268529
215,6059,California,Orange County,3155816,1558245,1597571,34.2,41.4,1.6,0.2,19.5,0.3,2021028,81851,570,37603,270,12.1,16.4,40.9,17.5,24.8,6.6,10.2,78.6,9.8,2.2,1.9,2.0,5.6,27.4,1560997,82.0,10.1,7.7,0.2,5.8,258306695416
103,4013,Arizona,Maricopa County,4155501,2055464,2100037,30.6,56.3,5.1,1.5,3.9,0.2,2778337,58580,350,30186,163,15.7,22.5,37.4,18.1,26.7,8.4,9.5,76.4,10.9,2.2,1.5,2.5,6.4,25.8,1929015,82.8,11.1,5.9,0.1,6.0,243429248580
222,6073,California,San Diego County,3283665,1651147,1632518,33.4,46.2,4.7,0.4,11.5,0.4,2207599,70588,617,34350,192,13.3,17.1,41.3,19.6,23.0,7.6,8.4,76.0,8.9,3.1,2.9,2.2,7.0,25.7,1536073,77.8,14.2,7.8,0.2,7.1,231787345020
228,6085,California,Santa Clara County,1911226,963317,947909,26.1,32.6,2.4,0.2,34.9,0.3,1162019,106761,858,48689,418,8.6,9.7,52.1,15.1,18.6,6.3,7.9,75.1,10.5,4.4,2.1,3.1,4.8,28.0,967294,84.7,9.3,5.9,0.1,5.7,204044398986
2969,53033,Washington,King County,2118119,1058695,1059424,9.5,61.4,6.0,0.5,16.5,0.8,1459791,83571,617,46316,363,10.2,12.3,51.0,15.0,19.9,5.5,8.5,63.0,9.6,13.1,5.1,2.7,6.5,29.1,1140353,82.0,12.0,5.8,0.2,5.0,177013322949
1225,25017,Massachusetts,Middlesex County,1582857,773112,809745,7.7,73.3,4.8,0.1,11.2,0.0,1110166,92878,735,47616,380,8.2,9.3,55.0,14.3,19.1,5.3,6.3,68.1,6.9,12.1,5.0,2.4,5.6,30.3,862873,82.9,10.7,6.2,0.1,4.8,147012592446
1868,36081,New York,Queens County,2339280,1134031,1205249,28.0,25.4,17.2,0.2,25.1,0.0,1413921,62008,471,28814,216,13.7,18.2,33.3,24.6,23.3,8.2,10.7,31.8,6.7,51.7,5.8,1.5,2.6,43.5,1142363,80.1,13.7,6.1,0.1,6.9,145054074240


Ah, this is more like it.  California regains top billing. But Maricopa County beating out Silicon Valley is anti-climatic.  

What we want to do is make nothing less than a Pandas clone! 

## First step: identifying the main concepts

The first step in the implementation consists in identifying the main concepts that need translating into "objects with behavior". In our case, this is easy: the concepts are the tables, with their rows and columns. 

## Table Representation

We can represent tables both in row-wise, or column-wise, fashion. 
In a row-wise fashion, the table is represented as a list of rows; each row is then (for instance) a dictionary mapping the column name to the column value in that row. 
In column-wise fashion, we represent the table as a dictionary mapping each column name to the column content; each column is then a list (or perhaps numpy array) of values. 

We prefer the column-wise representation, for three reasons. 
First, the representation is more efficient: it takes less space to have a dictionary of arrays than an array of dictionaries, since dictionaries are less efficient in representation than arrays. 
Second, the type of elements in a table column is homogeneous, opening the way to various optimizations both in storing, and in processing, the column elements. 
Third, it is often useful to do operations on columns, computing for instance a column as a function of two existing columns.  By using a column-wise organization, we can leverage numpy for implementing the column operations. 

The next choice consists in deciding whether we need a special object type for representing each column, or whether we can simply use a list, or perhaps a numpy array. 
We argue for representing columns as objects, for two reasons. 
First, a good principle is, in case of doubt, choose the most general representation. 
An object representation allows us to store the content of the column, alongside any additional metadata we may wish to associate with it. 
Second, an object representation allows us to implement operations on columns, by implementing the `__add__`, `__mul__`, and other [arithmetic methods](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types). 

## Column Representation

Let us thus focus first on how to represent a single column. 
The following are our design requirements: 

* Columns of numbers are very common, and we wish to implement them in such a way that we can easily serialize them (convert them to/from strings), and perform numerical operations on them.  
* Aside from numerical columns, we wish to be able to store columns consisting of arbitrary types (such as strings, datetime objects, and more). 
* Many columns have missing values, and we want to be able to easily represent and correctly process such missing types. 

This suggests the following representation: 

* Numeric columns are represented via numpy, so that arithmetic operations on them are implemented efficiently.  Missing values are implemented via NaN (Not A Number, in the [IEEE Floating Point Standard](https://en.wikipedia.org/wiki/IEEE_754)).  This is a somewhat imprecise mapping: a missing value, and a NaN, have slightly different semantics (a missing value vs. the undetermined result of an operation), but it will do for our purposes, as missing values and NaNs propagate similarly in operations. 
* Other columns are represented as lists.  We do not use numpy for non-numeric types, because numpy has all sort of restrictions for non-numeric types (for instance, strings are fixed maximal length, so once we assigned values to a column of a table, we would not be able to update particular values with longer strings). 

### Initialization

How do we initialize a column?  The most obvious ways is by passing a list of values.  In addition to the values, we will also pass a type hint, which is a tuple consisting of: 

* `None` (the python type, not a tuple): indicating that the type has to be inferred. 
* `('other',)` : indicating a non-numpy type,
* `('numpy', 'd')` : indicating [numpy type](https://numpy.org/doc/stable/user/basics.types.html) 'd'. 

If the type is auto-inferred, we use the following algorithm: 

* If all the elements are either None or numeric, we use the float type of numpy; 
* otherwise, we use `('other',)`.

The implementation is as follows.

In [200]:
import numpy as np
from numbers import Number

class Column(object):

    def __init__(self, element_list, type_hint=None):
        """Creates a column.
        @param element_list: list of elements with which to initialize the column.
            If a Column is passed in, then a copy of the Column is constructed.
        @param type_hint: hint for the type of column elements.  The type hint
        can be:
         - 'other', in which case, a list is used;
         - ('numpy', t), in which case, numpy is used for the column, and
            the numpy type of column elements is t.
         - None.  In this case, we try to use the best type; if all elements of
            the list are None or an empty string or numbers, then a numpy array
            is used, otherwise, a list.
        Internally, a Column object has two important fields:
        - d : this is the data.
            You can tell the format of self.d by asking:
            isinstance(self.d, np.ndarray) : True if it's a numpy array, false
            otherwise.
        """
        # Determines the type.
        if isinstance(element_list, Column):
            # We clone columns.  It's a good principle to always allow cloning by passing
            # an element of the class to the initializer.
            if isinstance(element_list.d, np.ndarray):
                self.d = np.array(element_list.d)
            else:
                self.d = list(element_list.d)
        elif isinstance(element_list, np.ndarray):
            # If you are kind enough to give me an ndarray, I build a Column
            # around it.
            self.d = np.array(element_list) # Clone to prevent modifications.
        # Otherwise, we must create the Column from a list-like type.
        elif type_hint is None:
            # We need to infer the type.
            # If it's all numeric, then we use a numeric type.
            if all([(x is None) or isinstance(x, Number) or x == '' for x in element_list]):
                # If there are None, we have to convert them in np.NaN
                self.d = np.array([(np.NaN if (x is None or x == '') else x) for x in element_list])
            else:
                # We use list() here both to create a copy of the list,
                # and to ensure we really get a list.
                self.d = list(element_list)
        elif type_hint[0] == 'other':
            self.d = list(element_list)
        elif type_hint[0] == 'numpy':
            self.d = np.array(element_list, dtype=type_hint[1])
        else:
            raise NotImplementedError()

    def __repr__(self):
        """We produce an abbreviated representation, since a full one might be very large."""
        if len(self.d) < 21:
            return "Column: " + repr(list(self.d))
        else:
            s = "Column: ["
            for x in self.d[:10]:
                s += repr(x) + ', '
            s += '...'
            for x in self.d[-10:]:
                s += ', ' + repr(x)
            s += ']'
            return s

    def __eq__(self, other):
        """Unless we define equality, object equality would be the notion of equality."""
        if isinstance(self.d, np.ndarray) and isinstance(other.d, np.ndarray):
            return np.allclose(self.d, other.d, equal_nan=True)
        else:
            return self.d == other.d



Let's try our implementation. 

In [201]:
Column([1, 2, 3])


Column: [1, 2, 3]

In [202]:
Column(list(range(30)) + [None, 1])


Column: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..., 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, nan, 1.0]

In [203]:
a = Column(range(10))
b = Column(a)
a == b


True

In [204]:
 Column(['I', 'Like', "Bananas"]) == Column(['I', 'Like', "Bananas"])


True

## Operations on Columns

What are the fundamental operations on columns? For a column `c`:  

* Get the length of a column via `len(c)`. 
* Get the `i`-th element of `c` via `c[i]`
* Set the `i`-th element of `c` via `c[i] = ...`

These are straightforward to implement, by just remanding to the corresponding methods of the internal data representation.

In [205]:
Column.__len__ = lambda self : len(self.d)

Column.__getitem__ = lambda self, i : self.d[i]

def column_setitem(self, i, x):
    self.d[i] = x
Column.__setitem__ = column_setitem


We would like also to implement iteration.

In [206]:
def column_iter(self):
    for x in self.d:
        yield x
Column.__iter__ = column_iter


### Exercise: Arithmetic operations between columns

**Exercise:** Define numerical operations +, -, /, * for columns.  These work as the usual arithmetic operations for numpy types.  For other types, just use the operations +, -, /, * between elements, hoping that they are defined; if you get an error, you insert `None` for that value, so that the operation itself never breaks.  This is done so that if you add, for instance, a fist name to a last name to obtain a complete name, if the last name of a person is `None`, you get `None` for the full name of the person, rather than an error.  It is rather annoying to process a large amount of data and fail because a few pieces of information are missing.

In [207]:
### Question 1: Arithmetic operations

class IncompatibleColumns(Exception):
    """We raise this exception if you try to combine columns that are incompatible,
    e.g, they are of different length."""
    pass

def column_binary_op(self, other, op):
    if len(self) != len(other):
        raise IncompatibleColumns()
    # YOUR CODE HERE
    r=[]
    for i in range(len(self)):
        r.append(op(self[i], other[i]))
    return Column(r)



def column_add(self, other):
    return column_binary_op(self, other, lambda x, y: x + y)

# Define similarly the other operators.  Think at what to do for __neg__,
# which is a unary operator; there are many solutions, all ok.

# YOUR CODE HERE
def column_sub(self, other):
    return column_binary_op(self, other, lambda x, y: x - y)
def column_mul(self, other):
    return column_binary_op(self, other, lambda x, y: x * y)
def column_div(self, other):
    return column_binary_op(self, other, lambda x, y: x / y)
def column_neg(self, other):
    df['r']=df['self'] +df['other']
    return r


Column.__add__ = column_add
Column.__sub__ = column_sub
Column.__mul__ = column_mul
Column.__div__ = column_div
Column.__neg__ = column_neg


In [208]:
#@title Let's define a testing helper.

def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)
    print("Success")


In [209]:
### Question 1: 10 points

### Tests for operations between numpy columns.

a = Column([1, 2, 3])
b = Column(np.array([5, 6, 7]))
check_equal(a + b, Column([6, 8, 10]))
check_equal(a * b, Column([5, 12, 21]))



Success
Success


In [210]:
### Question 1: 10 points

### Tests for operations between generic columns.

a = Column(["a", "b", "c"])
b = Column(["d", "e", "f"])
check_equal(a + b, Column(['ad', 'be', 'cf']))



Success


**Exercise:** Define an `apply` method, so that for a column c, and a function f on its elements, 

    c.apply(f)

returns a new column where `f` has been applied to each element of the column.  For example, if we define 

def f(x):
    return x + 1

then 

    c.apply(f)

would return a new column, where each element of the previous column has been incremented by 1.  This is similar (but better!) to the [`apply` function of Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html); at least we don't need that confusing axis=1 argument every single time.  

In [211]:
### Question 2: Define an apply function for columns.

def column_apply(self, f):
    # This is a one-liner.
    # YOUR CODE HERE
    return Column([f(x) for x in self.d])

Column.apply = column_apply


In [212]:
### Question 2, 10 points: Tests for apply function

# It works for numbers...
c = Column([1, 2, 3, 4])
def f(x):
    return x * x
check_equal(c.apply(f), Column([1, 4, 9, 16]))

# And it works for strings too!
c = Column("i like to go to london".split())
def f(x):
    return x.capitalize()
c.apply(f)
check_equal(c.apply(f), Column(['I', 'Like', 'To', 'Go', 'To', 'London']))


Success
Success


## Tables

Our class Table will be a subclass of OrderedDict. 
[OrderedDict](https://docs.python.org/3.7/library/collections.html#collections.OrderedDict) is a Python dictionary that remembers the order in which items were inserted.  

This is not an ideal implementation, and it would perhaps be better to keep the order of the columns in a separate list. 
But at least, when we read a csv file, we can preserve the column order. 

In [213]:
from collections import OrderedDict as MyOrderedDictionary
# If you have a good implementation of OrderedDictionary, you could import it via:
# import OrderedDictionary as MyOrderedDictionary

class Table(MyOrderedDictionary):

    def __init__(self, *args, **kwargs):
        # We initialize the ordered dictionary.
        super().__init__(*args, **kwargs)
        # And we check that column names and types have appropriate values.
        # Remember: here, .items() are the (column_name, column) pairs.
        for k, v in self.items():
            if not isinstance(k, str):
                raise TypeError("Column names must be of type str")
            if not isinstance(v, Column):
                # Converts to column
                self[k] = Column(v)
        if len(self) > 0:
            # Checks that all columns have the same length.
            lens = [(k, len(v)) for k, v in self.items()]
            for k, l in lens[1:]:
                if l != lens[0][1]:
                    raise TypeError("Column %s has length %d but column %s has length %d" %
                                    (lens[0][0], lens[0][1], k, l))

    @property
    def num_rows(self):
        """The number of rows of the table.  It would be nicer to redefine
        __len__ to return the number of rows, but as Table is an extension of
        a dictionary, we do not wish to subvert the normal meaning of __len__."""
        for v in self.values():
            return len(v)


In [214]:
t = Table(city=['London', 'New York', 'San Francisco'], population=[8908081, 8398748, 883305], csa=[None, 22679948, 9666055])
t


Table([('city', Column: ['London', 'New York', 'San Francisco']),
       ('population', Column: [8908081, 8398748, 883305]),
       ('csa', Column: [nan, 22679948.0, 9666055.0])])

### Table iteration

Let's define a row iterator which yields at each iteration a row, represented as a dictionary mapping from column names to column values at that row.  We will let you do it. 


In [215]:
### Question 3: Table iteration

def iter_table(self):
    # YOUR CODE HERE
    for i in range(self.num_rows):
        r={}
        for n in self.keys():
            r[n]=self[n][i]
        yield r

Table.__iter__ = iter_table


In [216]:
t = Table(city=['London', 'New York', 'San Francisco'],
          population=[8908081, 8398748, 883305],
          csa=[None, 22679948, 9666055])
for r in t:
    print(r)


{'city': 'London', 'population': 8908081, 'csa': nan}
{'city': 'New York', 'population': 8398748, 'csa': 22679948.0}
{'city': 'San Francisco', 'population': 883305, 'csa': 9666055.0}


In [217]:
### Question 3, 10 points: Tests for table iteration

t = Table(city=['London', 'New York', 'San Francisco'],
          population=[8908081, 8398748, 883305],
          csa=[34, 22679948, 9666055])
rows = [r for r in t]
check_equal(rows[0], {'city': 'London', 'population': 8908081, 'csa': 34})
check_equal(rows[1], {'city': 'New York', 'population': 8398748, 'csa': 22679948.0})
check_equal(rows[2], {'city': 'San Francisco', 'population': 883305, 'csa': 9666055.0})



Success
Success
Success


## Accessing and slicing rows

Given a table, we are interested in _slicing_ some of its rows. 

If we have a list `l = ['a', 'b', 'c', 'd']`, we can slice the list via `l[2:]` and obtain only the elements starting at position 2, or `l[:3]` and obtain the first three elements of `l`, namely, `['a', 'b', 'c']`. 
In Python, slicing is customarily used with the `[]` notation, and therefore, with `__getitem__`.  But in our tables, the square-brackets `[]` and `__getitem__` are already used to access columns.  Thus, we define `loc` as a way to access rows by their index.  Given a table t, 

    t.loc[4]

will give row 4 (as a dictionary mapping column names to column values, as for iteration), and 

    t.loc[:5]

will give the first five rows, as a new table.  This behavior is dishomogeneous: the return type of `loc` depends on its input type.  However, it is natural to represent a single row as a dictionary, so that: 

    for row in t:
        do_something(row)

and 

    for i in range(t.num_rows):
        do_something(t.loc[i])

behave in the same way.  Slicing is often used to obtain smaller datasets from larger ones, and thus, it is natural to return a new Table as a result.

How do we implement this?  The implementation is a little advanced, but also very instructive. 
Ranges such as `:5` in `[:5]` are [slice objects](https://docs.python.org/3/library/functions.html#slice) in Python. 
Such slice objects represent the set of indices in the slice.
Now, we wish that we could write a method loc, that could take as input both an integer and a slice.  In this way, we could write 

    t.loc(3)

and 

    t.loc(:6)

The problem is that in Python, the only way of constructing a slice object using the nice `:` syntax is to use it in square brackets, such as `[:5]`, not as a function argument as in `(:5)`.  
How can we make such square-bracket syntax work?  Remember that the square-bracket syntax is associated, in Python, with the `__getitem__` method.  Therefore, for `t.loc[:5]` to work, `t.loc` must be an object that supports `__getitem__`. 
So the implementation is as follows: 

* We define a `TableSlicer` class.  Its objects are initialized with a reference to the table, and they support the `__getitem__` method, so they can easily get a slice as argument.  The `__getitem__` method will check the type of its argument.  If it is an integer, it will return the given row.  If it is a [slice](https://docs.python.org/3/library/functions.html#slice), it will return a new table,  with the specified rows.

* We define the `loc` _property_ to return a `TableSlicer` object for the class.  We let `loc` be a _property_, rather than a method, so we can write

    t.loc[:5]

rather than 

    t.loc()[:5]

In summary, the expression `t.loc[:5]` can be read as follows:

* `t.loc` returns a `TableSlicer` for `t`;
* `[:5]` causes the `__getitem__` method of the `TableSlicer` object to be called with the slice `:5`, and will return the table consisting of the first five rows of the original table.

This is a lot of words, but the code is actually quite concise.

We note that there is a difference: 

    t.loc[4]

returns a dictionary header to values of row 4. 
Instead, 

    t.loc[4:5]

returns a table containing a single row, corresponding to row 4. 

We think this is a useful difference, and we keep this behavior.

In [218]:
class TableSlicer(object):

    def __init__(self, table):
        """Initializes a table slicer for the given table."""
        self.table = table

    def __getitem__(self, i):
        """Returns either the row i of the table, as a dictionary
        mapping column name to column content, or if i is a slice,
        returns a new table corresponding to the slice of rows."""
        headers = list(self.table.keys())
        if isinstance(i, int):
            return {k: self.table[k][i] for k in headers}
        elif isinstance(i, slice):
            # Here we trust that both lists and np.arrays support slices,
            # which they do.
            return Table({k: self.table[k][i] for k in headers})
        else:
            raise TypeError("loc called with non-integer, non-slice argument")

def table_loc(self):
    """This returns a slicer object, so we can write something like t.loc[i] with the [i]
    using the __getitem__ method of t.loc"""
    return TableSlicer(self)

# We use the property decorator here, so we can write t.loc[i] instead of t.loc()[i]
Table.loc = property(table_loc)


### Displaying tables

A table looks like a table only if we ... display it as a table!  Let us provide a visualization via the [`_repr_html_`](https://ipython.readthedocs.io/en/stable/config/integrating.html#rich-display) method of IPython.

In [219]:
from IPython.display import HTML, display
import tabulate # See https://pypi.org/project/tabulate/

def table_repr(self, max_rows=20):
    """Displays a table.  At most max_rows are displayed; if max_rows = 0, then
    all rows are displayed."""
    headers = list(self.keys())
    if max_rows == 0 or self.num_rows <= max_rows:
        rows = [[r[k] for k in headers] for r in self]
    else:
        rows_first = [[r[k] for k in headers] for r in self.loc[:max_rows // 2]]
        rows_last  = [[r[k] for k in headers] for r in self.loc[- max_rows // 2:]]
        mid_row = ['...' for k in headers]
        rows = rows_first + [mid_row] + rows_last
    # Displays the table.
    display(HTML(tabulate.tabulate(rows, headers=headers, tablefmt='html')))
    return "" # __repr__ must return a string...

# The _repr_html_ method is the method that is
Table._repr_html_ = table_repr


In [220]:
Table(city=['London', 'New York', 'San Francisco'], population=[8908081, 8398748, 883305], csa=[None, 22679948, 9666055])


city,population,csa
London,8.90808e+06,nan
New York,8.39875e+06,2.26799e+07
San Francisco,883305,9.66606e+06


Table([('city', Column: ['London', 'New York', 'San Francisco']),
       ('population', Column: [8908081, 8398748, 883305]),
       ('csa', Column: [nan, 22679948.0, 9666055.0])])

In [221]:
from IPython.display import HTML, display
import tabulate # See https://pypi.org/project/tabulate/

table = [["London", 8908081],
         ["New York", 8398748],
         ["San Francisco", 883305]]
display(HTML(tabulate.tabulate(table, headers=["City", "Population"], tablefmt='html')))


City,Population
London,8908081
New York,8398748
San Francisco,883305


In [222]:
t = Table(city=['London', 'New York', 'San Francisco'], population=[8908081, 8398748, 883305], csa=[None, 22679948, 9666055])
t.loc[2]


{'city': 'San Francisco', 'csa': 9666055.0, 'population': 883305}

**Exercise:** Write `head`, `tail`, `idx`, and `sample` methods:

* `head(n)` should return a new table consisting of the first `n` rows of the table; 
* `tail(n)` should return a new table consisting of the last `n` rows of the table;
* `idx(l)` should return a new table consisting of the rows whose indices are in the list `l`.  So if `l = [2, 4, 5]`, the new table should consist of rows 2, 4, 5 of the table. 
* `sample(n)` should return a new table consisting of `n` rows sampled at random among those of the table. 

_Hint:_ The head and tail methods are trivial to implement, using the above .loc one.  The one method slightly more difficult is `idx`.  Once you implement `idx`, it is easy to implement `sample` by looking at the documentation for [random.sample](https://docs.python.org/3/library/random.html#random.sample). 

In [223]:
### Question 4: Define head, tail, and sample methods.

# This is used in the sample method.
import random

def table_head(self, n=5):
    """Returns the top n elements (or less, if the table is shorter) of the
    table, as a new table."""
    # YOUR CODE HERE
    return self.loc[:n]

def table_tail(self, n=5):
    """Returns the bottom n elements of the table as a new table."""
    # YOUR CODE HERE
    return self.loc[5-n:]

def table_idx(self, indices):
    """Given a list of indices, returns a new table consisting of the
    rows of the table with the specified indices."""
    # YOUR CODE HERE
    r=Table(self)
    for i in self.keys():
        r[i]=[]
        for n in indices:
            r[i].append(self[i][n])
    return r
    

def table_sample(self, n=5):
    """Returns a sample of length n of the table's rows, as a new table."""
    # YOUR CODE HERE
    r=[]
    f=[]
    for i in range(self.num_rows):
        r.append(i)
    f=random.sample(r,n)
    return table_idx(self,f)

Table.head = table_head
Table.tail = table_tail
Table.idx = table_idx
Table.sample = table_sample


Let's test the methods. 

In [224]:
### Question 4, 10 points: Tests for `head` and `tail`
t = Table(city=['London', 'New York City', 'San Francisco', 'Santa Cruz', 'Mountain View'],
          population=[8908081, 8398748, 883305, 82739, 65541],
          area=[1572, 1212, 600, 31, 33])

t1 = t.head()
t2 = t.tail()
check_equal(t1, t2)
check_equal(t, t1)
t1 = t.head(2)
check_equal(t1.num_rows, 2)
check_equal(t.loc[0], t1.loc[0])
check_equal(t.loc[1], t1.loc[1])
t2 = t.tail(2)
check_equal(t.loc[3], t2.loc[0])
check_equal(t.loc[4], t2.loc[1])



Success
Success
Success
Success
Success
Success
Success


In [225]:
### Question 4, 10 points: Tests for `idx`

t = Table(city=['London', 'New York City', 'San Francisco', 'Santa Cruz', 'Mountain View'],
          population=[8908081, 8398748, 883305, 82739, 65541],
          area=[1572, 1212, 600, 31, 33])

idxs = [0, 1, 3]
tt = t.idx(idxs)
check_equal(tt.loc[0], {'city': 'London', 'population': 8908081, 'area': 1572})
check_equal(tt.loc[1], {'city': 'New York City', 'population': 8398748, 'area': 1212})
check_equal(tt.loc[2], {'city': 'Santa Cruz', 'population': 82739, 'area': 31})



Success
Success
Success


### Implementing `nlargest`

We are almost done with our reimplementation of Pandas.  We just need one more thing: tne `nlargest` method, so that 

    t.nlargest(10, "column_name")

returns a new table containing the 10 rows of `t` that have the largest values in the column named `column_name`. 
The catch is that you have to do it _without_ modifying (e.g., sorting) the original table.  If you want to do it in a truly sophisticated way (and you should, if you can), you should use the [`heapq` module](https://docs.python.org/3/library/heapq.html).  

_Hint:_ the solution consists of just a few lines of code, using the `idx` method defined previously, and `heapq.nlargest`. 

In [226]:
### Question 5: Implementing `nlargest`

import heapq # In case you need it.

def table_nlargest(self, n, column_name):
    # YOUR CODE HERE
    f=heapq.nlargest(n,enumerate(self[column_name]),key=lambda x:x[1])
    r=[]
    for i,n in f:
        r.append(i)
    g=self.idx(r)
    return g

Table.nlargest = table_nlargest


In [227]:
### Question 5: 10 points: Tests for `nlargest`

t = Table(city=['Milan', 'New York City', 'Napoli', 'San Francisco', 'Santa Cruz', 'Mountain View'],
          population=[12, 8398748, 883305, 34, 82739, 65541],
          area=[34, 1212, 600, 3100, 12, 33])
tt = t.nlargest(2, 'population')
check_equal(tt.loc[0], {'city': 'New York City', 'population': 8398748, 'area': 1212})
check_equal(tt.loc[1], {'city': 'Napoli', 'population': 883305, 'area': 600})
tt = t.nlargest(2, 'area')
check_equal(tt.loc[0], {'city': 'San Francisco', 'population': 34, 'area': 3100})
check_equal(tt.loc[1], {'city': 'New York City', 'population': 8398748, 'area': 1212})
check_equal(t.loc[0], {'city': 'Milan', 'population': 12, 'area': 34})
check_equal(tt.num_rows, 2)
check_equal(t.num_rows, 6)



Success
Success
Success
Success
Success
Success
Success


### Reading csv files into Tables

Data for data science often comes in [csv](https://docs.python.org/3/library/csv.html) form.  We will write two constructors: 

* `from_csv`: creates a Table from a csv file, assuming that the first row contains the table names. 
* `from_csv_url`: creates a Table from a URL, from which a csv file can be downloaded. 

Obviously, the second constructor will be implemented in terms of the first one.  As the Table does not yet exist when these methods are called, the methods are static, constructed with the Python [`staticmethod`](https://docs.python.org/3/library/functions.html#staticmethod) decorator.

In [228]:
import csv, requests, codecs
from contextlib import closing

def csv_convert(x):
    """Tries to convert x to a number, if it can be converted.
    This implementation is a bit naive; a better implementation would try to
    also convert dates to date objects."""
    try:
        return int(x)
    except:
        pass
    try:
        return float(x)
    except:
        return x

def table_from_csv(csv_file):
    """Creates a table from a csv file."""
    d = MyOrderedDictionary()
    headers = None
    for row in csv.reader(csv_file):
        if headers is None:
            headers = list(row)
            d = MyOrderedDictionary()
            for k in headers:
                d[k] = []
        else:
            # We pad the row, so we can read it all.
            prow = list(row)
            if len(row) < len(headers):
                prow += [None] * (len(headers) - len(row))
            prow = row[:len(headers)]
            for i, k in enumerate(headers):
                d[k].append(csv_convert(row[i]))
    return Table(d)

Table.from_csv = staticmethod(table_from_csv)

def table_from_csv_url(url):
    """Returns a table from a given url."""
    with closing(requests.get(url, stream=True)) as r:
        return Table.from_csv(codecs.iterdecode(r.iter_lines(), 'utf-8'))

Table.from_csv_url = staticmethod(table_from_csv_url)


## Examining the demographics dataset with our Tables

We can now carry out the same analysis we did with Pandas, using our own `Tables` class. 
Let's load and display the data. 



In [229]:
t = Table.from_csv_url("https://storage.googleapis.com/lucadealfaro-share/acs2017_county_data.csv")
t


CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9
1011,Alabama,Bullock County,10478,5616,4862,0.3,21.6,75.6,1.0,0.7,0.0,8212,29655,5376,20856,2355,28.5,50.4,19.7,17.1,18.6,14.0,30.6,73.1,15.7,0.3,6.2,1.7,3.0,29.8,4290,81.4,13.6,5.0,0.0,12.1
1013,Alabama,Butler County,20126,9416,10710,0.3,52.2,44.7,0.1,1.1,0.0,15459,36326,2701,19004,943,24.4,34.8,26.9,17.3,18.5,11.6,25.7,83.6,12.6,0.0,0.9,0.9,2.0,23.2,7727,79.1,15.3,5.3,0.3,7.6
1015,Alabama,Calhoun County,115527,55593,59934,3.6,72.7,20.4,0.2,1.0,0.0,88383,43686,1491,23638,793,18.6,26.6,29.0,17.5,23.7,10.4,19.4,85.0,9.2,0.2,1.3,1.1,3.2,24.8,47392,74.9,19.9,5.1,0.1,10.1
1017,Alabama,Chambers County,33895,16320,17575,2.2,56.2,39.3,0.3,1.0,0.0,26259,37342,2011,22002,1205,18.8,29.1,24.3,13.5,23.0,11.6,27.6,87.1,9.7,0.2,0.6,0.5,2.0,23.6,14527,84.5,11.8,3.7,0.0,6.4
1019,Alabama,Cherokee County,25855,12862,12993,1.6,91.8,5.0,0.5,0.1,0.0,20620,40041,2316,23010,1354,16.1,20.0,28.8,14.8,18.1,11.9,26.5,85.0,12.1,0.4,0.3,0.3,2.0,26.5,9879,74.8,17.1,8.1,0.0,5.3


Table([('CountyId',
        Column: [1001, 1003, 1005, 1007, 1009, 1011, 1013, 1015, 1017, 1019, ..., 72135, 72137, 72139, 72141, 72143, 72145, 72147, 72149, 72151, 72153]),
       ('State',
        Column: ['Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', ..., 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico', 'Puerto Rico']),
       ('County',
        Column: ['Autauga County', 'Baldwin County', 'Barbour County', 'Bibb County', 'Blount County', 'Bullock County', 'Butler County', 'Calhoun County', 'Chambers County', 'Cherokee County', ..., 'Toa Alta Municipio', 'Toa Baja Municipio', 'Trujillo Alto Municipio', 'Utuado Municipio', 'Vega Alta Municipio', 'Vega Baja Municipio', 'Vieques Municipio', 'Villalba Municipio', 'Yabucoa Municipio', 'Yauco Municipio']),
       ('TotalPop',
        Column: [55036, 203360, 26201, 22580, 57667, 10478, 20126, 115527, 33895, 25855, ..., 74169, 81905, 69478, 30209, 38589, 54754, 8931, 23659, 35025, 37585]),
       ('Men',
        Column: [26899, 99527, 13976, 12251, 28490, 5616, 9416, 55593, 16320, 12862, ..., 35682, 38453, 32747, 14686, 18371, 26269, 4351, 11510, 16984, 18052]),
       ('Women',
        Column: [28137, 103833, 12225, 10329, 29177, 4862, 10710, 59934, 17575, 12993, ..., 38487, 43452, 36731, 15523, 20218, 28485, 4580, 12149, 18041, 19533]),
       ('Hispanic',
        Column: [2.7, 4.4, 4.2, 2.4, 9.0, 0.3, 0.3, 3.6, 2.2, 1.6, ..., 99.1, 99.3, 99.3, 99.5, 98.6, 96.7, 95.7, 99.7, 99.9, 99.8]),
       ('White',
        Column: [75.4, 83.1, 45.7, 74.6, 87.4, 21.6, 52.2, 72.7, 56.2, 91.8, ..., 0.4, 0.5, 0.6, 0.4, 0.9, 3.1, 4.0, 0.2, 0.1, 0.2]),
       ('Black',
        Column: [18.9, 9.5, 47.8, 22.0, 1.5, 75.6, 44.7, 20.4, 39.3, 5.0, ..., 0.1, 0.1, 0.0, 0.1, 0.0, 0.1, 0.0, 0.1, 0.0, 0.0]),
       ('Native',
        Column: [0.3, 0.8, 0.2, 0.4, 0.3, 1.0, 0.1, 0.2, 0.3, 0.5, ..., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
       ('Asian',
        Column: [0.9, 0.7, 0.6, 0.0, 0.1, 0.7, 1.1, 1.0, 1.0, 0.1, ..., 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
       ('Pacific',
        Column: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
       ('VotingAgeCitizen',
        Column: [41016, 155376, 20269, 17662, 42513, 8212, 15459, 88383, 26259, 20620, ..., 56601, 63417, 53545, 23755, 29549, 42838, 7045, 18053, 27523, 29763]),
       ('Income',
        Column: [55317, 52562, 33368, 43404, 47412, 29655, 36326, 43686, 37342, 40041, ..., 29672, 24583, 31202, 15931, 18053, 18900, 16261, 19893, 15586, 14451]),
       ('IncomeErr',
        Column: [2838, 1348, 2551, 3431, 2630, 5376, 2701, 1491, 2011, 2316, ..., 1584, 1174, 1312, 1104, 1218, 1219, 2414, 1935, 1467, 1371]),
       ('IncomePerCap',
        Column: [27824, 29364, 17561, 20911, 22021, 20856, 19004, 23638, 22002, 23010, ..., 14162, 12240, 15703, 8140, 10492, 10197, 11136, 10449, 8672, 8124]),
       ('IncomePerCapErr',
        Column: [2024, 735, 798, 1889, 850, 2355, 943, 793, 1205, 1354, ..., 698, 403, 879, 411, 922, 576, 1459, 1619, 702, 445]),
       ('Poverty',
        Column: [13.7, 11.8, 27.2, 15.2, 15.6, 28.5, 24.4, 18.6, 18.8, 16.1, ..., 29.8, 35.8, 30.4, 53.0, 50.6, 43.8, 36.8, 50.0, 52.4, 50.4]),
       ('ChildPoverty',
        Column: [20.1, 16.1, 44.9, 26.6, 25.4, 50.4, 34.8, 26.6, 29.1, 20.0, ..., 33.7, 44.2, 41.5, 65.6, 64.3, 49.4, 68.2, 67.9, 62.1, 58.2]),
       ('Professional',
        Column: [35.3, 35.7, 25.0, 24.4, 28.5, 19.7, 26.9, 29.0, 24.3, 28.8, ..., 36.0, 28.5, 36.7, 24.7, 31.4, 28.6, 20.9, 22.5, 27.7, 30.4]),
       ('Service',
        Column: [18.0, 18.2, 16.8, 17.6, 12.9, 17.1, 17.3, 17.5, 13.5, 14.8, ..., 14.7, 21.0, 16.9, 24.4, 19.7, 20.2, 38.4, 21.2, 26.0, 20.2]),
       ('Office',
        Column: [23.2, 25.6, 22.6, 19.7, 23.3, 18.6, 18.5, 23.7, 23.0, 18.1, ..., 33.9, 32.1, 30.9, 30.2,

Here is Virginia again, with its patrician counties: 

In [230]:
t.nlargest(10, "Income")


CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
51107,Virginia,Loudoun County,374558,185575,188983,13.5,57.7,7.2,0.2,17.1,0.1,231130,129588,2111,50456,581,3.7,3.6,57.3,13.2,19.9,4.5,5.1,77.2,9.2,3.7,1.7,1.1,7,33.7,201528,78.7,16.8,4.4,0.1,3.8
51059,Virginia,Fairfax County,1.142e+06,565593,576411,16.2,51.7,9.3,0.1,18.8,0,724787,117515,1314,52976,419,6.1,7.9,56.6,14.7,18.4,5.7,4.7,71.1,9.4,9.6,1.9,1.7,6.3,32.1,609859,73.6,20.8,5.5,0.1,4.5
24027,Maryland,Howard County,312495,152843,159652,6.5,54.1,18.1,0.2,17.3,0,212310,115576,1799,51045,772,5.2,5.8,60.9,11.8,18.1,4.1,5.1,81.2,7.2,3.8,1,1.1,5.7,30.9,167493,73,22.7,4.2,0.1,4.2
51610,Virginia,Falls Church city,13843,6801,7042,10.6,72,4.5,0,9.3,0,9598,114795,7917,65510,4014,2.9,2.4,67.3,11.7,16,2.1,2.9,60.4,7.6,20.1,2.4,2.8,6.7,28.9,7672,64.6,28.5,6.9,0,4.2
51013,Virginia,Arlington County,229534,114403,115131,15.5,62.3,8.5,0.3,10.2,0.1,162937,112138,2139,67061,879,8.2,10,69,10.5,13.8,4.1,2.7,53.4,6.2,26,5,2.8,6.6,28.6,143798,70.8,24.9,4.2,0.2,3.2
8035,Colorado,Douglas County,320940,159804,161136,8.3,83.3,1.1,0.1,4.4,0.1,222455,111154,1484,48725,653,3.6,3.8,53.7,11.6,25,4.6,5,78,6.7,1.9,1,1.4,11,27.6,169625,83.3,11.6,5.1,0.1,3.3
34019,New Jersey,Hunterdon County,125717,62901,62816,6.2,86.3,2.5,0.1,3.8,0,95691,110969,2182,54200,1457,4.5,4.9,51.7,12.9,23.2,6.7,5.5,80.7,5.5,2.6,1.8,0.7,8.7,33.9,66382,79.5,13.7,6.6,0.1,4.4
35028,New Mexico,Los Alamos County,18031,9135,8896,17.2,72.9,0.3,1,6,0,13223,110190,6543,52125,2359,5.1,5.4,67.7,12.6,13,4.2,2.5,76.5,9.9,3.3,3.9,3.6,2.9,15.7,9122,61.6,34.3,3.8,0.3,3.7
34027,New Jersey,Morris County,498847,244480,254367,13,72,3,0,10.1,0,354414,107034,1215,53491,803,4.6,4.8,51,12.9,23.9,5.3,6.9,79.2,6.8,5.4,2,0.9,5.6,30.9,262461,82.9,11.6,5.4,0.1,5.2
51600,Virginia,Fairfax city,23580,11621,11959,16.9,57.9,4.9,0.1,16,0.1,15316,106870,5914,46489,2037,7.8,6.2,53,18.1,18.6,6.8,3.6,69.6,7.6,10.1,4.5,1.1,7.2,31.2,12834,74.7,20.8,4.3,0.1,4.1


Table([('CountyId',
        [51107, 51059, 24027, 51610, 51013, 8035, 34019, 35028, 34027, 51600]),
       ('State',
        ['Virginia',
         'Virginia',
         'Maryland',
         'Virginia',
         'Virginia',
         'Colorado',
         'New Jersey',
         'New Mexico',
         'New Jersey',
         'Virginia']),
       ('County',
        ['Loudoun County',
         'Fairfax County',
         'Howard County',
         'Falls Church city',
         'Arlington County',
         'Douglas County',
         'Hunterdon County',
         'Los Alamos County',
         'Morris County',
         'Fairfax city']),
       ('TotalPop',
        [374558,
         1142004,
         312495,
         13843,
         229534,
         320940,
         125717,
         18031,
         498847,
         23580]),
       ('Men',
        [185575,
         565593,
         152843,
         6801,
         114403,
         159804,
         62901,
         9135,
         244480,
         11621]),
       ('Women',
        [188983,
         576411,
         159652,
         7042,
         115131,
         161136,
         62816,
         8896,
         254367,
         11959]),
       ('Hispanic', [13.5, 16.2, 6.5, 10.6, 15.5, 8.3, 6.2, 17.2, 13.0, 16.9]),
       ('White', [57.7, 51.7, 54.1, 72.0, 62.3, 83.3, 86.3, 72.9, 72.0, 57.9]),
       ('Black', [7.2, 9.3, 18.1, 4.5, 8.5, 1.1, 2.5, 0.3, 3.0, 4.9]),
       ('Native', [0.2, 0.1, 0.2, 0.0, 0.3, 0.1, 0.1, 1.0, 0.0, 0.1]),
       ('Asian', [17.1, 18.8, 17.3, 9.3, 10.2, 4.4, 3.8, 6.0, 10.1, 16.0]),
       ('Pacific', [0.1, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.1]),
       ('VotingAgeCitizen',
        [231130,
         724787,
         212310,
         9598,
         162937,
         222455,
         95691,
         13223,
         354414,
         15316]),
       ('Income',
        [129588,
         117515,
         115576,
         114795,
         112138,
         111154,
         110969,
         110190,
         107034,
         106870]),
       ('IncomeErr',
        [2111, 1314, 1799, 7917, 2139, 1484, 2182, 6543, 1215, 5914]),
       ('IncomePerCap',
        [50456,
         52976,
         51045,
         65510,
         67061,
         48725,
         54200,
         52125,
         53491,
         46489]),
       ('IncomePerCapErr',
        [581, 419, 772, 4014, 879, 653, 1457, 2359, 803, 2037]),
       ('Poverty', [3.7, 6.1, 5.2, 2.9, 8.2, 3.6, 4.5, 5.1, 4.6, 7.8]),
       ('ChildPoverty', [3.6, 7.9, 5.8, 2.4, 10.0, 3.8, 4.9, 5.4, 4.8, 6.2]),
       ('Professional',
        [57.3, 56.6, 60.9, 67.3, 69.0, 53.7, 51.7, 67.7, 51.0, 53.0]),
       ('Service',
        [13.2, 14.7, 11.8, 11.7, 10.5, 11.6, 12.9, 12.6, 12.9, 18.1]),
       ('Office',
        [19.9, 18.4, 18.1, 16.0, 13.8, 25.0, 23.2, 13.0, 23.9, 18.6]),
       ('Construction', [4.5, 5.7, 4.1, 2.1, 4.1, 4.6, 6.7, 4.2, 5.3, 6.8]),
       ('Production', [5.1, 4.7, 5.1, 2.9, 2.7, 5.0, 5.5, 2.5, 6.9, 3.6]),
       ('Drive', [77.2, 71.1, 81.2, 60.4, 53.4, 78.0, 80.7, 76.5, 79.2, 69.6]),
       ('Carpool', [9.2, 9.4, 7.2, 7.6, 6.2, 6.7, 5.5, 9.9, 6.8, 7.6]),
       ('Transit', [3.7, 9.6, 3.8, 20.1, 26.0, 1.9, 2.6, 3.3, 5.4, 10.1]),
       ('Walk', [1.7, 1.9, 1.0, 2.4, 5.0, 1.0, 1.8, 3.9, 2.0, 4.5]),
       ('OtherTransp', [1.1, 1.7, 1.1, 2.8, 2.8, 1.4, 0.7, 3.6, 0.9, 1.1]),
       ('WorkAtHome', [7.0, 6.3, 5.7, 6.7, 6.6, 11.0, 8.7, 2.9, 5.6, 7.2]),
       ('MeanCommute',
        [33.7, 32.1, 30.9, 28.9, 28.6, 27.6, 33.9, 15.7, 30.9, 31.2]),
       ('Employed',
        [201528,
         609859,
         167493,
         7672,
         143798,
         169625,
         66382,
         9122,
         262461,
         12834]),
       ('PrivateWork',
        [78.7, 73.6, 73.0, 64.6, 70.8, 83.3, 79.5, 61.6, 82.9, 74.7]),
       ('PublicWork',
        [16.8, 20.8, 22.7, 28.5, 24.9, 11.6, 13.7, 34.3, 11.6, 20.8]),
       ('SelfEmployed', [4.4, 5.5, 4.2, 6.9, 4.2, 5.1, 6.6, 3.8, 5.4, 4.3]),
       ('FamilyWork', [0.1, 0.1, 0.1, 0.0,

Let's compute a new column, giving each county's purchasing power. 

In [231]:
t["PursePower"] = t["TotalPop"] * t["Income"]


And here's again the role of California counties in purchasing power. 

In [232]:
t.nlargest(10, "PursePower")


CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,PursePower
6037,California,Los Angeles County,1.01057e+07,4.97964e+06,5.12608e+06,48.4,26.5,7.9,0.2,14.3,0.2,6.21828e+06,61015,262,30798,129,17,24,36.4,19,24.1,7.6,12.8,73.7,9.6,6.3,2.7,2.3,5.3,30.9,4.80582e+06,79.3,11.2,9.3,0.2,7.8,6.16601e+11
17031,Illinois,Cook County,5.23854e+06,2.5407e+06,2.69784e+06,25.1,42.7,23.4,0.1,6.9,0,3.53844e+06,59426,380,33722,200,15.9,23,39.3,17.9,23.6,6,13.1,61.7,8.1,18.9,4.4,2.4,4.5,32.9,2.52144e+06,84.2,11.1,4.6,0.1,8.7,3.11306e+11
48201,Texas,Harris County,4.52552e+06,2.25106e+06,2.27446e+06,42.2,30.6,18.5,0.2,6.8,0.1,2.60504e+06,57791,503,30856,222,16.8,25,35.4,17.4,22.9,11.5,12.8,79.3,10.9,2.7,1.5,1.9,3.7,28.9,2.18039e+06,83.4,9.8,6.6,0.2,6.4,2.61534e+11
6059,California,Orange County,3.15582e+06,1.55824e+06,1.59757e+06,34.2,41.4,1.6,0.2,19.5,0.3,2.02103e+06,81851,570,37603,270,12.1,16.4,40.9,17.5,24.8,6.6,10.2,78.6,9.8,2.2,1.9,2,5.6,27.4,1.561e+06,82,10.1,7.7,0.2,5.8,2.58307e+11
4013,Arizona,Maricopa County,4.1555e+06,2.05546e+06,2.10004e+06,30.6,56.3,5.1,1.5,3.9,0.2,2.77834e+06,58580,350,30186,163,15.7,22.5,37.4,18.1,26.7,8.4,9.5,76.4,10.9,2.2,1.5,2.5,6.4,25.8,1.92902e+06,82.8,11.1,5.9,0.1,6,2.43429e+11
6073,California,San Diego County,3.28366e+06,1.65115e+06,1.63252e+06,33.4,46.2,4.7,0.4,11.5,0.4,2.2076e+06,70588,617,34350,192,13.3,17.1,41.3,19.6,23,7.6,8.4,76,8.9,3.1,2.9,2.2,7,25.7,1.53607e+06,77.8,14.2,7.8,0.2,7.1,2.31787e+11
6085,California,Santa Clara County,1.91123e+06,963317,947909,26.1,32.6,2.4,0.2,34.9,0.3,1.16202e+06,106761,858,48689,418,8.6,9.7,52.1,15.1,18.6,6.3,7.9,75.1,10.5,4.4,2.1,3.1,4.8,28,967294,84.7,9.3,5.9,0.1,5.7,2.04044e+11
53033,Washington,King County,2.11812e+06,1.0587e+06,1.05942e+06,9.5,61.4,6,0.5,16.5,0.8,1.45979e+06,83571,617,46316,363,10.2,12.3,51,15,19.9,5.5,8.5,63,9.6,13.1,5.1,2.7,6.5,29.1,1.14035e+06,82,12,5.8,0.2,5,1.77013e+11
25017,Massachusetts,Middlesex County,1.58286e+06,773112,809745,7.7,73.3,4.8,0.1,11.2,0,1.11017e+06,92878,735,47616,380,8.2,9.3,55,14.3,19.1,5.3,6.3,68.1,6.9,12.1,5,2.4,5.6,30.3,862873,82.9,10.7,6.2,0.1,4.8,1.47013e+11
36081,New York,Queens County,2.33928e+06,1.13403e+06,1.20525e+06,28,25.4,17.2,0.2,25.1,0,1.41392e+06,62008,471,28814,216,13.7,18.2,33.3,24.6,23.3,8.2,10.7,31.8,6.7,51.7,5.8,1.5,2.6,43.5,1.14236e+06,80.1,13.7,6.1,0.1,6.9,1.45054e+11


Table([('CountyId',
        [6037, 17031, 48201, 6059, 4013, 6073, 6085, 53033, 25017, 36081]),
       ('State',
        ['California',
         'Illinois',
         'Texas',
         'California',
         'Arizona',
         'California',
         'California',
         'Washington',
         'Massachusetts',
         'New York']),
       ('County',
        ['Los Angeles County',
         'Cook County',
         'Harris County',
         'Orange County',
         'Maricopa County',
         'San Diego County',
         'Santa Clara County',
         'King County',
         'Middlesex County',
         'Queens County']),
       ('TotalPop',
        [10105722,
         5238541,
         4525519,
         3155816,
         4155501,
         3283665,
         1911226,
         2118119,
         1582857,
         2339280]),
       ('Men',
        [4979641,
         2540704,
         2251060,
         1558245,
         2055464,
         1651147,
         963317,
         1058695,
         773112,
         1134031]),
       ('Women',
        [5126081,
         2697837,
         2274459,
         1597571,
         2100037,
         1632518,
         947909,
         1059424,
         809745,
         1205249]),
       ('Hispanic',
        [48.4, 25.1, 42.2, 34.2, 30.6, 33.4, 26.1, 9.5, 7.7, 28.0]),
       ('White', [26.5, 42.7, 30.6, 41.4, 56.3, 46.2, 32.6, 61.4, 73.3, 25.4]),
       ('Black', [7.9, 23.4, 18.5, 1.6, 5.1, 4.7, 2.4, 6.0, 4.8, 17.2]),
       ('Native', [0.2, 0.1, 0.2, 0.2, 1.5, 0.4, 0.2, 0.5, 0.1, 0.2]),
       ('Asian', [14.3, 6.9, 6.8, 19.5, 3.9, 11.5, 34.9, 16.5, 11.2, 25.1]),
       ('Pacific', [0.2, 0.0, 0.1, 0.3, 0.2, 0.4, 0.3, 0.8, 0.0, 0.0]),
       ('VotingAgeCitizen',
        [6218279,
         3538445,
         2605043,
         2021028,
         2778337,
         2207599,
         1162019,
         1459791,
         1110166,
         1413921]),
       ('Income',
        [61015,
         59426,
         57791,
         81851,
         58580,
         70588,
         106761,
         83571,
         92878,
         62008]),
       ('IncomeErr', [262, 380, 503, 570, 350, 617, 858, 617, 735, 471]),
       ('IncomePerCap',
        [30798,
         33722,
         30856,
         37603,
         30186,
         34350,
         48689,
         46316,
         47616,
         28814]),
       ('IncomePerCapErr', [129, 200, 222, 270, 163, 192, 418, 363, 380, 216]),
       ('Poverty', [17.0, 15.9, 16.8, 12.1, 15.7, 13.3, 8.6, 10.2, 8.2, 13.7]),
       ('ChildPoverty',
        [24.0, 23.0, 25.0, 16.4, 22.5, 17.1, 9.7, 12.3, 9.3, 18.2]),
       ('Professional',
        [36.4, 39.3, 35.4, 40.9, 37.4, 41.3, 52.1, 51.0, 55.0, 33.3]),
       ('Service',
        [19.0, 17.9, 17.4, 17.5, 18.1, 19.6, 15.1, 15.0, 14.3, 24.6]),
       ('Office',
        [24.1, 23.6, 22.9, 24.8, 26.7, 23.0, 18.6, 19.9, 19.1, 23.3]),
       ('Construction', [7.6, 6.0, 11.5, 6.6, 8.4, 7.6, 6.3, 5.5, 5.3, 8.2]),
       ('Production', [12.8, 13.1, 12.8, 10.2, 9.5, 8.4, 7.9, 8.5, 6.3, 10.7]),
       ('Drive', [73.7, 61.7, 79.3, 78.6, 76.4, 76.0, 75.1, 63.0, 68.1, 31.8]),
       ('Carpool', [9.6, 8.1, 10.9, 9.8, 10.9, 8.9, 10.5, 9.6, 6.9, 6.7]),
       ('Transit', [6.3, 18.9, 2.7, 2.2, 2.2, 3.1, 4.4, 13.1, 12.1, 51.7]),
       ('Walk', [2.7, 4.4, 1.5, 1.9, 1.5, 2.9, 2.1, 5.1, 5.0, 5.8]),
       ('OtherTransp', [2.3, 2.4, 1.9, 2.0, 2.5, 2.2, 3.1, 2.7, 2.4, 1.5]),
       ('WorkAtHome', [5.3, 4.5, 3.7, 5.6, 6.4, 7.0, 4.8, 6.5, 5.6, 2.6]),
       ('MeanCommute',
        [30.9, 32.9, 28.9, 27.4, 25.8, 25.7, 28.0, 29.1, 30.3, 43.5]),
       ('Employed',
        [4805817,
         2521437,
         2180392,
         1560997,
         1929015,
         1536073,
         967294,
         1140353,
         862873,
         1142363]),
       ('PrivateWork',
        [79.3, 84.2, 83.4, 82.0, 82.8, 77.8, 84.7, 82.0, 82.9, 80.1]),
       ('PublicWork',
        [11.2, 11.1, 9.8, 10.1, 11.1, 14.2, 9.3, 12.0, 10.7, 13.7]),
       ('SelfEmployed', [9.3, 4.6, 6.6, 7.7, 5.

See that it is not hard, after all, to produce a clone of Pandas in a few well-chosen hundred lines of code? 